# Transformer FEMCA Demonstration
The purpose of this script it to demonstrate the impact of using a new consequence model for the Zone Sub Transfomer FEMCA

Requestors: danielle.fisher@essentialenergy.com.au; ken.holder@essentialenergy.com.au

Author: gavin.treseder@essentialenergy.com.au

# Step 1 - Load the packages
Load python packages to aid with data analysis and visualisation


In [7]:
import numpy as np 
import pandas as pd
import math

from scipy.integrate import quad
import scipy.stats as ss

import tkinter as tk
from tkinter.filedialog import askopenfilename

## Step 2 - Load the data
Load the data from the "Tx FMECA to consequence" file into a dataframe and check the contents are as expected

In [8]:
try:
    file_name
except:
    file_name = askopenfilename(initialdir=os.getcwd())

In [9]:
# Read the file in and remove the first row
df_fmeca = pd.read_excel(file_name, header = 1)

df_fmeca.head()

,TMC code,Category,Equipment,TMC,Band,Function,Functional Failure,Failure Cause,No.,Local Effect,...,Optimal Interval,Protective systems,Protective Population,Protective MTBF,Protective Repair,Protective Failures,Asset Consequence Model,Level Consequence Model,Notes,Unnamed: 54
0,TX 01 01 01,ZS Power Transformer,Zone and Subtrans. Generic Transformers.,TX010101 Zone and Subtrans. Generic Transformers.,Essential Energy,02) Provides insulation of live equipment.,a) Does not provide insulation of live equipment.,EXTERNAL BUSHING damaged porcelain due to flas...,2,"Flashover,",...,0.0,1,0.0,0,0,0.0,Bushing,Major,NaN,NaN
1,TX 01 01 01,ZS Power Transformer,Zone and Subtrans. Generic Transformers.,TX010101 Zone and Subtrans. Generic Transformers.,Essential Energy,02) Provides insulation of live equipment.,a) Does not provide insulation of live equipment.,EXTERNAL BUSHING damaged porcelain due to vand...,3,"Flashover,",...,0.0,1,0.0,0,0,0.0,Bushing,Major,NaN,NaN
2,TX 01 01 01,ZS Power Transformer,Zone and Subtrans. Generic Transformers.,TX010101 Zone and Subtrans. Generic Transformers.,Essential Energy,01) Transformation of power from primary to se...,a) Does not transform power from primary to se...,BUSHINGS faulty connection due to corrosion / ...,4,Overheated Connection,...,631.0,1,0.0,0,0,0.0,NaN,NaN,hot joint bushing,NaN
3,TX 01 01 01,ZS Power Transformer,Zone and Subtrans. Generic Transformers.,TX010101 Zone and Subtrans. Generic Transformers.,Essential Energy,02) Provides insulation of live equipment.,a) Does not provide insulation of live equipment.,EXTERNAL BUSHING internal fault due to oil los...,5,"Flashover, possible secondary damage and flyin...",...,0.0,1,0.0,0,0,0.0,Bushing,Major,NaN,NaN
4,TX 01 01 01,ZS Power Transformer,Zone and Subtrans. Generic Transformers.,TX010101 Zone and Subtrans. Generic Transformers.,Essential Energy,02) Provides insulation of live equipment.,a) Does not provide insulation of live equipment.,EXTERNAL BUSHING low resistance due to surface...,6,"Flashover, possible secondary damage and flyin...",...,0.0,1,0.0,0,0,0.0,Bushing,Major,NaN,NaN


In [10]:
filter_cols = ['Failure Cause', 'Proposed Task', 'Test', 'Scheduled Task', 'Scheduled Interval', 'Asset Consequence Model', "Level Consequence Model"]

df_tasks = df_fmeca[filter_cols]
df_tasks.head()

,Failure Cause,Proposed Task,Test,Scheduled Task,Scheduled Interval,Asset Consequence Model,Level Consequence Model
0,EXTERNAL BUSHING damaged porcelain due to flas...,Examine all transformer bushing surfaces for d...,NaN,Examine all transformer bushing surfaces for d...,ZS Inspect,Bushing,Major
1,EXTERNAL BUSHING damaged porcelain due to vand...,Examine all transformer bushing surfaces for d...,NaN,Examine all transformer bushing surfaces for d...,ZS Inspect,Bushing,Major
2,BUSHINGS faulty connection due to corrosion / ...,Examine bushing connection condition on load u...,NaN,Examine bushing connection condition on load u...,Infrared Scan,NaN,NaN
3,EXTERNAL BUSHING internal fault due to oil los...,Check bushing sight glass for correct oil level.,NaN,Check bushing sight glass for correct oil level.,ZS Inspect,Bushing,Major
4,EXTERNAL BUSHING low resistance due to surface...,Examine transformer bushing surface for contam...,NaN,Examine transformer bushing surface for contam...,6yr Maint.,Bushing,Major


In [13]:
filter_cols = ["Operation Loss:","CF Interval", "MTBF", "Examination Time (m):", "Exam Success", "Setup Cost", "Exam Cost", "Planned Repair Cost", "Call Out, Secondary Damage", 	"Unplanned Failure Cost"]

df_params = df_fmeca[filter_cols]
df_params.head()

,Operation Loss:,CF Interval,MTBF,Examination Time (m):,Exam Success,Setup Cost,Exam Cost,Planned Repair Cost,"Call Out, Secondary Damage",Unplanned Failure Cost
0,0,0,169567,0,0.0,0.0,0.0,0,0,0
1,0,0,1196389,0,0.0,0.0,0.0,0,0,0
2,120000,1460,34067,10,95.0,15.0,0.0,2500,142500,20000
3,0,0,0,0,0.0,0.0,0.0,0,0,0
4,12000,0,0,0,NaN,NaN,0.0,0,12000,0


## Estimate the optimum interval

Using the same method from MMIR

In [43]:
def optimum_interval(operation_loss, cf_interval, mtbf, exam_time, exam_success, setup_cost, exam_cost, planned_repair_cost, call_out, unplanned_failure_cost):

    # Scale some of the units
    inspect_success = exam_success / 100

    inspect_cost = exam_cost + setup_cost # Inpsection Cost
    penalty = operation_loss + call_out # Penalty
    T = cf_interval

    optimum = math.log(((-mtbf / T) * inspect_cost) / (penalty * math.log(1 - inspect_success))) / math.log(1 - inspect_success)

    optimum_interval = T / optimum

    return optimum_interval

In [45]:
optimum_interval(*tuple(df_params.iloc[8]))

1852.2650777102656

In [46]:
dict(df_params.iloc[8])

{'Operation Loss:': 120000.0,
 'CF Interval': 3650.0,
 'MTBF': 41948.0,
 'Examination Time (m):': 15.0,
 'Exam Success': 98.0,
 'Setup Cost': 55.0,
 'Exam Cost': 110.0,
 'Planned Repair Cost': 40000.0,
 'Call Out, Secondary Damage': 960000.0,
 'Unplanned Failure Cost': 800000.0}